In [4]:
import pandas as pd
import numpy as np
from pathlib import Path

In [5]:
# Load data_daily files
data_folder = Path("data")

data_daily_A = pd.read_csv(data_folder / 'data_daily_A.csv', index_col=0, parse_dates=True)
data_daily_B = pd.read_csv(data_folder / 'data_daily_B.csv', index_col=0, parse_dates=True)
data_daily_C = pd.read_csv(data_folder / 'data_daily_C.csv', index_col=0, parse_dates=True)


print("Data loaded successfully!")
print(f"data_daily_A: {data_daily_A.shape}")
print(f"data_daily_B: {data_daily_B.shape}")
print(f"data_daily_C: {data_daily_C.shape}")

Data loaded successfully!
data_daily_A: (755, 2)
data_daily_B: (739, 2)
data_daily_C: (737, 2)


In [ ]:
# A

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels import api as sm
from xgboost import XGBRegressor
from pygam import ExpectileGAM
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# Set seed for reproducibility
np.random.seed(42)

In [ ]:
# SLIDING WINDOW OLS REGRESSION RESULTS FOR PRODUCT A
# Window: 180 days training, 30 days testing

data = data_daily_A.copy()
dates = data.index.to_numpy()

# Feature and target
feature = 'log_sell_price_A'
target = 'log_sales_A'

# Sliding window parameters
train_window = 180
test_window = 7

results = []

for i in range(len(dates) - train_window - test_window + 1):
    train_start_idx = i
    train_end_idx = i + train_window - 1
    test_start_idx = i + train_window
    test_end_idx = i + train_window + test_window - 1
    
    train_data = data.iloc[train_start_idx:train_end_idx+1]
    test_data = data.iloc[test_start_idx:test_end_idx+1]
    
    if len(train_data) == 0 or len(test_data) == 0:
        continue
    
    X_train = train_data[[feature]]
    y_train = train_data[target]
    X_test = test_data[[feature]]
    y_test = test_data[target]
    
    # OLS Regression
    X_train_sm = sm.add_constant(X_train)
    model = sm.OLS(y_train, X_train_sm).fit()
    
    X_test_sm = sm.add_constant(X_test)
    y_pred = model.predict(X_test_sm)
    
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        'Window': i + 1,
        'Train_Start': dates[train_start_idx],
        'Train_End': dates[train_end_idx],
        'Test_Start': dates[test_start_idx],
        'Test_End': dates[test_end_idx],
        'Train_Size': len(train_data),
        'Test_Size': len(test_data),
        'Coef': model.params[1],
        'Intercept': model.params[0],
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2
    })

results_df = pd.DataFrame(results)
print("SLIDING WINDOW OLS REGRESSION RESULTS FOR PRODUCT A")
print(f"Windows generated: {len(results_df)}")
print(f"Mean MAE: {results_df['MAE'].mean():.4f}, Mean RMSE: {results_df['RMSE'].mean():.4f}, Mean R²: {results_df['R2'].mean():.4f}")

In [ ]:
# Visualize OLS Results for Product A
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(results_df['MAE'], marker='o', label='MAE', alpha=0.7)
axes[0].set_ylabel('MAE')
axes[0].set_xlabel('Window')
axes[0].set_title('Mean Absolute Error Over Sliding Windows')
axes[0].grid(True, alpha=0.3)

axes[1].plot(results_df['RMSE'], marker='o', label='RMSE', alpha=0.7, color='orange')
axes[1].set_ylabel('RMSE')
axes[1].set_xlabel('Window')
axes[1].set_title('Root Mean Squared Error Over Sliding Windows')
axes[1].grid(True, alpha=0.3)

axes[2].plot(results_df['R2'], marker='o', label='R²', alpha=0.7, color='green')
axes[2].set_ylabel('R²')
axes[2].set_xlabel('Window')
axes[2].set_title('R² Score Over Sliding Windows')
axes[2].grid(True, alpha=0.3)
axes[2].axhline(y=0, color='red', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

In [ ]:
# SLIDING WINDOW XGBoost REGRESSION FOR PRODUCT A
results_xgb = []

for i in range(len(dates) - train_window - test_window + 1):
    train_start_idx = i
    train_end_idx = i + train_window - 1
    test_start_idx = i + train_window
    test_end_idx = i + train_window + test_window - 1
    
    train_data = data.iloc[train_start_idx:train_end_idx+1]
    test_data = data.iloc[test_start_idx:test_end_idx+1]
    
    if len(train_data) == 0 or len(test_data) == 0:
        continue
    
    X_train = train_data[[feature]].values
    y_train = train_data[target].values
    X_test = test_data[[feature]].values
    y_test = test_data[target].values
    
    # XGBoost
    model_xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=42)
    model_xgb.fit(X_train, y_train)
    y_pred_xgb = model_xgb.predict(X_test)
    
    mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
    rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
    r2_xgb = r2_score(y_test, y_pred_xgb)
    
    results_xgb.append({
        'Window': i + 1,
        'Train_Start': dates[train_start_idx],
        'Train_End': dates[train_end_idx],
        'Test_Start': dates[test_start_idx],
        'Test_End': dates[test_end_idx],
        'Train_Size': len(train_data),
        'Test_Size': len(test_data),
        'MAE': mae_xgb,
        'RMSE': rmse_xgb,
        'R2': r2_xgb
    })

results_xgb_df = pd.DataFrame(results_xgb)
print("SLIDING WINDOW XGBoost REGRESSION RESULTS FOR PRODUCT A")
print(f"Windows generated: {len(results_xgb_df)}")
print(f"Mean MAE: {results_xgb_df['MAE'].mean():.4f}, Mean RMSE: {results_xgb_df['RMSE'].mean():.4f}, Mean R²: {results_xgb_df['R2'].mean():.4f}")

In [ ]:
# Visualize XGBoost Results for Product A
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(results_xgb_df['MAE'], marker='o', label='MAE', alpha=0.7)
axes[0].set_ylabel('MAE')
axes[0].set_xlabel('Window')
axes[0].set_title('Mean Absolute Error Over Sliding Windows (XGBoost)')
axes[0].grid(True, alpha=0.3)

axes[1].plot(results_xgb_df['RMSE'], marker='o', label='RMSE', alpha=0.7, color='orange')
axes[1].set_ylabel('RMSE')
axes[1].set_xlabel('Window')
axes[1].set_title('Root Mean Squared Error Over Sliding Windows (XGBoost)')
axes[1].grid(True, alpha=0.3)

axes[2].plot(results_xgb_df['R2'], marker='o', label='R²', alpha=0.7, color='green')
axes[2].set_ylabel('R²')
axes[2].set_xlabel('Window')
axes[2].set_title('R² Score Over Sliding Windows (XGBoost)')
axes[2].grid(True, alpha=0.3)
axes[2].axhline(y=0, color='red', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

In [ ]:
# SLIDING WINDOW PyGAM QUANTILE REGRESSION FOR PRODUCT A
quantiles = [0.025, 0.5, 0.975]
results_pygam_dfs = {}

for q in quantiles:
    results_pygam = []
    
    for i in range(len(dates) - train_window - test_window + 1):
        train_start_idx = i
        train_end_idx = i + train_window - 1
        test_start_idx = i + train_window
        test_end_idx = i + train_window + test_window - 1
        
        train_data = data.iloc[train_start_idx:train_end_idx+1]
        test_data = data.iloc[test_start_idx:test_end_idx+1]
        
        if len(train_data) == 0 or len(test_data) == 0:
            continue
        
        X_train = train_data[[feature]].values
        y_train = train_data[target].values
        X_test = test_data[[feature]].values
        y_test = test_data[target].values
        
        # PyGAM
        model_pygam = ExpectileGAM(expectile=q)
        model_pygam.fit(X_train, y_train)
        y_pred_pygam = model_pygam.predict(X_test)
        
        mae_pygam = mean_absolute_error(y_test, y_pred_pygam)
        rmse_pygam = np.sqrt(mean_squared_error(y_test, y_pred_pygam))
        r2_pygam = r2_score(y_test, y_pred_pygam)
        
        results_pygam.append({
            'Window': i + 1,
            'Train_Start': dates[train_start_idx],
            'Train_End': dates[train_end_idx],
            'Test_Start': dates[test_start_idx],
            'Test_End': dates[test_end_idx],
            'Train_Size': len(train_data),
            'Test_Size': len(test_data),
            'MAE': mae_pygam,
            'RMSE': rmse_pygam,
            'R2': r2_pygam
        })
    
    results_pygam_dfs[q] = pd.DataFrame(results_pygam)
    print(f"SLIDING WINDOW PyGAM (Q={q}) REGRESSION FOR PRODUCT A")
    print(f"Windows generated: {len(results_pygam_dfs[q])}")
    print(f"Mean MAE: {results_pygam_dfs[q]['MAE'].mean():.4f}, Mean RMSE: {results_pygam_dfs[q]['RMSE'].mean():.4f}, Mean R²: {results_pygam_dfs[q]['R2'].mean():.4f}\n")

In [ ]:
# Visualize PyGAM Results for Product A
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

q_median = results_pygam_dfs[0.5]

axes[0].plot(q_median['MAE'], marker='o', label='MAE', alpha=0.7)
axes[0].set_ylabel('MAE')
axes[0].set_xlabel('Window')
axes[0].set_title('Mean Absolute Error Over Sliding Windows (PyGAM)')
axes[0].grid(True, alpha=0.3)

axes[1].plot(q_median['RMSE'], marker='o', label='RMSE', alpha=0.7, color='orange')
axes[1].set_ylabel('RMSE')
axes[1].set_xlabel('Window')
axes[1].set_title('Root Mean Squared Error Over Sliding Windows (PyGAM)')
axes[1].grid(True, alpha=0.3)

axes[2].plot(q_median['R2'], marker='o', label='R²', alpha=0.7, color='green')
axes[2].set_ylabel('R²')
axes[2].set_xlabel('Window')
axes[2].set_title('R² Score Over Sliding Windows (PyGAM)')
axes[2].grid(True, alpha=0.3)
axes[2].axhline(y=0, color='red', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

# B

In [ ]:
# SLIDING WINDOW REGRESSION FOR PRODUCT B (OLS, XGBoost, PyGAM)
data_B = data_daily_B.copy()
dates_B = data_B.index.to_numpy()

feature_B = 'log_sell_price_B'
target_B = 'log_sales_B'

results_B = []
results_B_xgb = []

# OLS and XGBoost loops
for i in range(len(dates_B) - train_window - test_window + 1):
    train_start_idx = i
    train_end_idx = i + train_window - 1
    test_start_idx = i + train_window
    test_end_idx = i + train_window + test_window - 1
    
    train_data = data_B.iloc[train_start_idx:train_end_idx+1]
    test_data = data_B.iloc[test_start_idx:test_end_idx+1]
    
    if len(train_data) == 0 or len(test_data) == 0:
        continue
    
    X_train = train_data[[feature_B]]
    y_train = train_data[target_B]
    X_test = test_data[[feature_B]]
    y_test = test_data[target_B]
    
    # OLS
    X_train_sm = sm.add_constant(X_train)
    model = sm.OLS(y_train, X_train_sm).fit()
    X_test_sm = sm.add_constant(X_test)
    y_pred = model.predict(X_test_sm)
    
    results_B.append({
        'Window': i + 1,
        'Train_Start': dates_B[train_start_idx],
        'Train_End': dates_B[train_end_idx],
        'Test_Start': dates_B[test_start_idx],
        'Test_End': dates_B[test_end_idx],
        'Train_Size': len(train_data),
        'Test_Size': len(test_data),
        'Coef': model.params[1],
        'Intercept': model.params[0],
        'MAE': mean_absolute_error(y_test, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
        'R2': r2_score(y_test, y_pred)
    })
    
    # XGBoost
    X_train_xgb = train_data[[feature_B]].values
    y_train_xgb = train_data[target_B].values
    X_test_xgb = test_data[[feature_B]].values
    y_test_xgb = test_data[target_B].values
    
    model_xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=42)
    model_xgb.fit(X_train_xgb, y_train_xgb)
    y_pred_xgb = model_xgb.predict(X_test_xgb)
    
    results_B_xgb.append({
        'Window': i + 1,
        'MAE': mean_absolute_error(y_test_xgb, y_pred_xgb),
        'RMSE': np.sqrt(mean_squared_error(y_test_xgb, y_pred_xgb)),
        'R2': r2_score(y_test_xgb, y_pred_xgb)
    })

results_B_df = pd.DataFrame(results_B)
results_B_xgb_df = pd.DataFrame(results_B_xgb)

print("SLIDING WINDOW REGRESSION RESULTS FOR PRODUCT B")
print(f"OLS - Windows: {len(results_B_df)}, Mean MAE: {results_B_df['MAE'].mean():.4f}, Mean R²: {results_B_df['R2'].mean():.4f}")
print(f"XGBoost - Windows: {len(results_B_xgb_df)}, Mean MAE: {results_B_xgb_df['MAE'].mean():.4f}, Mean R²: {results_B_xgb_df['R2'].mean():.4f}")

In [ ]:
# PyGAM for Product B
results_B_pygam_dfs = {}

for q in quantiles:
    results_B_pygam = []
    
    for i in range(len(dates_B) - train_window - test_window + 1):
        train_start_idx = i
        train_end_idx = i + train_window - 1
        test_start_idx = i + train_window
        test_end_idx = i + train_window + test_window - 1
        
        train_data = data_B.iloc[train_start_idx:train_end_idx+1]
        test_data = data_B.iloc[test_start_idx:test_end_idx+1]
        
        if len(train_data) == 0 or len(test_data) == 0:
            continue
        
        X_train = train_data[[feature_B]].values
        y_train = train_data[target_B].values
        X_test = test_data[[feature_B]].values
        y_test = test_data[target_B].values
        
        model_pygam = ExpectileGAM(expectile=q)
        model_pygam.fit(X_train, y_train)
        y_pred_pygam = model_pygam.predict(X_test)
        
        results_B_pygam.append({
            'Window': i + 1,
            'MAE': mean_absolute_error(y_test, y_pred_pygam),
            'RMSE': np.sqrt(mean_squared_error(y_test, y_pred_pygam)),
            'R2': r2_score(y_test, y_pred_pygam)
        })
    
    results_B_pygam_dfs[q] = pd.DataFrame(results_B_pygam)
    print(f"PyGAM (Q={q}) - Windows: {len(results_B_pygam_dfs[q])}, Mean MAE: {results_B_pygam_dfs[q]['MAE'].mean():.4f}, Mean R²: {results_B_pygam_dfs[q]['R2'].mean():.4f}")

# C

In [ ]:
# SLIDING WINDOW REGRESSION FOR PRODUCT C (OLS, XGBoost, PyGAM)
data_C = data_daily_C.copy()
dates_C = data_C.index.to_numpy()

feature_C = 'log_sell_price_C'
target_C = 'log_sales_C'

results_C = []
results_C_xgb = []

# OLS and XGBoost loops
for i in range(len(dates_C) - train_window - test_window + 1):
    train_start_idx = i
    train_end_idx = i + train_window - 1
    test_start_idx = i + train_window
    test_end_idx = i + train_window + test_window - 1
    
    train_data = data_C.iloc[train_start_idx:train_end_idx+1]
    test_data = data_C.iloc[test_start_idx:test_end_idx+1]
    
    if len(train_data) == 0 or len(test_data) == 0:
        continue
    
    X_train = train_data[[feature_C]]
    y_train = train_data[target_C]
    X_test = test_data[[feature_C]]
    y_test = test_data[target_C]
    
    # OLS
    X_train_sm = sm.add_constant(X_train)
    model = sm.OLS(y_train, X_train_sm).fit()
    X_test_sm = sm.add_constant(X_test)
    y_pred = model.predict(X_test_sm)
    
    results_C.append({
        'Window': i + 1,
        'Train_Start': dates_C[train_start_idx],
        'Train_End': dates_C[train_end_idx],
        'Test_Start': dates_C[test_start_idx],
        'Test_End': dates_C[test_end_idx],
        'Train_Size': len(train_data),
        'Test_Size': len(test_data),
        'Coef': model.params[1],
        'Intercept': model.params[0],
        'MAE': mean_absolute_error(y_test, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
        'R2': r2_score(y_test, y_pred)
    })
    
    # XGBoost
    X_train_xgb = train_data[[feature_C]].values
    y_train_xgb = train_data[target_C].values
    X_test_xgb = test_data[[feature_C]].values
    y_test_xgb = test_data[target_C].values
    
    model_xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=42)
    model_xgb.fit(X_train_xgb, y_train_xgb)
    y_pred_xgb = model_xgb.predict(X_test_xgb)
    
    results_C_xgb.append({
        'Window': i + 1,
        'MAE': mean_absolute_error(y_test_xgb, y_pred_xgb),
        'RMSE': np.sqrt(mean_squared_error(y_test_xgb, y_pred_xgb)),
        'R2': r2_score(y_test_xgb, y_pred_xgb)
    })

results_C_df = pd.DataFrame(results_C)
results_C_xgb_df = pd.DataFrame(results_C_xgb)

print("SLIDING WINDOW REGRESSION RESULTS FOR PRODUCT C")
print(f"OLS - Windows: {len(results_C_df)}, Mean MAE: {results_C_df['MAE'].mean():.4f}, Mean R²: {results_C_df['R2'].mean():.4f}")
print(f"XGBoost - Windows: {len(results_C_xgb_df)}, Mean MAE: {results_C_xgb_df['MAE'].mean():.4f}, Mean R²: {results_C_xgb_df['R2'].mean():.4f}")

In [ ]:
# PyGAM for Product C
results_C_pygam_dfs = {}

for q in quantiles:
    results_C_pygam = []
    
    for i in range(len(dates_C) - train_window - test_window + 1):
        train_start_idx = i
        train_end_idx = i + train_window - 1
        test_start_idx = i + train_window
        test_end_idx = i + train_window + test_window - 1
        
        train_data = data_C.iloc[train_start_idx:train_end_idx+1]
        test_data = data_C.iloc[test_start_idx:test_end_idx+1]
        
        if len(train_data) == 0 or len(test_data) == 0:
            continue
        
        X_train = train_data[[feature_C]].values
        y_train = train_data[target_C].values
        X_test = test_data[[feature_C]].values
        y_test = test_data[target_C].values
        
        model_pygam = ExpectileGAM(expectile=q)
        model_pygam.fit(X_train, y_train)
        y_pred_pygam = model_pygam.predict(X_test)
        
        results_C_pygam.append({
            'Window': i + 1,
            'MAE': mean_absolute_error(y_test, y_pred_pygam),
            'RMSE': np.sqrt(mean_squared_error(y_test, y_pred_pygam)),
            'R2': r2_score(y_test, y_pred_pygam)
        })
    
    results_C_pygam_dfs[q] = pd.DataFrame(results_C_pygam)
    print(f"PyGAM (Q={q}) - Windows: {len(results_C_pygam_dfs[q])}, Mean MAE: {results_C_pygam_dfs[q]['MAE'].mean():.4f}, Mean R²: {results_C_pygam_dfs[q]['R2'].mean():.4f}")

# Comparison & Summary

In [ ]:
# OLS Comparison across all products
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Compare MAE
for df, label in [(results_df, 'A'), (results_B_df, 'B'), (results_C_df, 'C')]:
    if len(df) > 0:
        axes[0].plot(df['MAE'], label=f'Product {label}', alpha=0.7)
axes[0].set_ylabel('MAE')
axes[0].set_xlabel('Window')
axes[0].set_title('OLS: Mean Absolute Error Over Sliding Windows')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Compare RMSE
for df, label in [(results_df, 'A'), (results_B_df, 'B'), (results_C_df, 'C')]:
    if len(df) > 0:
        axes[1].plot(df['RMSE'], label=f'Product {label}', alpha=0.7)
axes[1].set_ylabel('RMSE')
axes[1].set_xlabel('Window')
axes[1].set_title('OLS: Root Mean Squared Error Over Sliding Windows')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Compare R²
for df, label in [(results_df, 'A'), (results_B_df, 'B'), (results_C_df, 'C')]:
    if len(df) > 0:
        axes[2].plot(df['R2'], label=f'Product {label}', alpha=0.7)
axes[2].set_ylabel('R²')
axes[2].set_xlabel('Window')
axes[2].set_title('OLS: R² Score Over Sliding Windows')
axes[2].legend()
axes[2].grid(True, alpha=0.3)
axes[2].axhline(y=0, color='red', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

In [ ]:
# XGBoost Comparison across all products
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Compare MAE
for df, label in [(results_xgb_df, 'A'), (results_B_xgb_df, 'B'), (results_C_xgb_df, 'C')]:
    if len(df) > 0:
        axes[0].plot(df['MAE'], label=f'Product {label}', alpha=0.7)
axes[0].set_ylabel('MAE')
axes[0].set_xlabel('Window')
axes[0].set_title('XGBoost: Mean Absolute Error Over Sliding Windows')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Compare RMSE
for df, label in [(results_xgb_df, 'A'), (results_B_xgb_df, 'B'), (results_C_xgb_df, 'C')]:
    if len(df) > 0:
        axes[1].plot(df['RMSE'], label=f'Product {label}', alpha=0.7)
axes[1].set_ylabel('RMSE')
axes[1].set_xlabel('Window')
axes[1].set_title('XGBoost: Root Mean Squared Error Over Sliding Windows')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Compare R²
for df, label in [(results_xgb_df, 'A'), (results_B_xgb_df, 'B'), (results_C_xgb_df, 'C')]:
    if len(df) > 0:
        axes[2].plot(df['R2'], label=f'Product {label}', alpha=0.7)
axes[2].set_ylabel('R²')
axes[2].set_xlabel('Window')
axes[2].set_title('XGBoost: R² Score Over Sliding Windows')
axes[2].legend()
axes[2].grid(True, alpha=0.3)
axes[2].axhline(y=0, color='red', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

In [ ]:
# Summary Table
summary_data = []

for product, ols_df, xgb_df, pygam_dfs in [
    ('A', results_df, results_xgb_df, results_pygam_dfs),
    ('B', results_B_df, results_B_xgb_df, results_B_pygam_dfs),
    ('C', results_C_df, results_C_xgb_df, results_C_pygam_dfs)
]:
    # OLS
    if len(ols_df) > 0:
        summary_data.append({
            'Product': product,
            'Model': 'OLS',
            'Mean_MAE': ols_df['MAE'].mean(),
            'Mean_RMSE': ols_df['RMSE'].mean(),
            'Mean_R2': ols_df['R2'].mean()
        })
    
    # XGBoost
    if len(xgb_df) > 0:
        summary_data.append({
            'Product': product,
            'Model': 'XGBoost',
            'Mean_MAE': xgb_df['MAE'].mean(),
            'Mean_RMSE': xgb_df['RMSE'].mean(),
            'Mean_R2': xgb_df['R2'].mean()
        })
    
    # PyGAM (Q=0.5)
    if 0.5 in pygam_dfs and len(pygam_dfs[0.5]) > 0:
        pygam_df = pygam_dfs[0.5]
        summary_data.append({
            'Product': product,
            'Model': 'PyGAM (Q=0.5)',
            'Mean_MAE': pygam_df['MAE'].mean(),
            'Mean_RMSE': pygam_df['RMSE'].mean(),
            'Mean_R2': pygam_df['R2'].mean()
        })

summary_df = pd.DataFrame(summary_data)
print("\n=== SUMMARY TABLE (Daily Data) ===")
print(summary_df.to_string(index=False))